In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
from time import sleep

#navigator
def chrome():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless") #without graphical interface
    options.add_argument('--ignore-ssl-errors=yes')
    options.add_argument('--ignore-certificate-errors')
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    return driver
driver = chrome()
urls = []

#load website
website = 'https://annuairesante.ameli.fr/'
path = 'C:/Users/DELL/Desktop/MediConnect/chrome-win64/chrome-win64/chrome.exe'
driver.get(website)

#click liste professions

list_profession = driver.find_element("css selector", ".popin-autocomplete-ps-professions.popin-autocomplete-enable")
list_profession.click()


In [ ]:
indexes = driver.find_elements("xpath", "//li[starts-with(@id, 'index-')]")

with open("Informations_medecins.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)

    # Write header row
    writer.writerow(["Nom","Prenom", "Specialtée", "Numero de téléphone", "Clinique", "Rue", "Code Postal", "Ville"])

    for i in range(len(indexes)):
        try:
            #refresh website
            driver.get(website)
            list_profession = driver.find_element("css selector", ".popin-autocomplete-ps-professions.popin-autocomplete-enable")
            list_profession.click()    

            #waiting for the popup
            WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "popin-content"))  
                )
            WebDriverWait(driver, 2).until(
                    EC.presence_of_all_elements_located((By.XPATH, "//li[starts-with(@id, 'index-')]"))
                )


            #relocate to prevent stale exception (DOM)
            updated_indexes = driver.find_elements("xpath", "//li[starts-with(@id, 'index-')]")

            li = updated_indexes[i]

            link = li.find_element("tag name", "a")

            #scroll
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", link)

            full_text = link.get_attribute("textContent").strip()
            #print(full_text)

            driver.execute_script("arguments[0].click();", link)

            # wait for submit busston availability before clicking
            submit = WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.NAME, "submit_final"))
            )

            driver.execute_script("arguments[0].click();", submit)

            while True:
                list_professionnels = driver.find_elements("class name" , "liste-professionnel")

                for pro in list_professionnels:
                    professionnel = pro.find_element("class name" , "item-professionnel").find_element("class name" , "item-professionnel-inner")

                # extract each doctor id inside num
                num_spans = driver.find_elements("css selector", ".item-professionnel .num")
                #print(num_spans.text)

                #loop over elements and estract other info(specialite, adress,...)
                for num_s in num_spans:
                    try:
                        parent_div = num_s.find_element("xpath", "./ancestor::div[@class='item-professionnel']")
                        nom_pictos = parent_div.find_element("css selector", ".nom_pictos h2 a")
                        #print(nom_pictos.text)

                        last_name_element = nom_pictos.find_elements(By.CSS_SELECTOR, "strong")

                        # If <strong> exists, extract last_name; otherwise, default to "N/A"
                        last_name = last_name_element[0].text if last_name_element else "N/A"

                        firstname = nom_pictos.text.replace(last_name, "").strip()                 

                        #print(num_s.text)
                        #print(last_name)
                        #print(firstname)
                        elements = parent_div.find_elements("class name" , "elements")
                        
                        for element in elements:
                            specialite = element.find_element("css selector", ".item.left.specialite").text         
                            #print(specialite)
                            tel = element.find_elements("css selector", ".item.left.tel")  
                            t = tel[0].text if tel else "N/A"
                            #print(t)
                            addresses = element.find_elements("css selector" , ".item.left.adresse")                 
                            
                            for address in addresses:
                                # using innerHTML, to get everything
                                full_add = address.get_attribute("innerHTML")
                                lines = [line.strip() for line in full_add.split("<br>")]

                                if len(lines) == 3:
                                    clinique = lines[0] 
                                    rue = lines[1] 
                                    if " " in lines[2]:
                                        code_postal, ville = lines[2].split(" ", 1) 
                                    else:
                                        code_postal, ville = (lines[2], "") 

                                elif len(lines) == 2: 
                                    clinique = "" 
                                    rue = lines[0]
                                    if " " in lines[1]:
                                        code_postal, ville = lines[1].split(" ", 1)
                                    else:
                                        code_postal, ville = (lines[1], "") 

                                #print(clinique)
                                #print(rue)
                                #print(code_postal)
                                #print(ville)
                                writer.writerow([ last_name, firstname, specialite, t, clinique, rue, code_postal, ville])
                                
                    except Exception as e:
                        print("Error processing:", e)
                                
                try: 
                    next_button = WebDriverWait(driver, 2).until(
                        EC.element_to_be_clickable((By.XPATH, "//a[img[@alt='Page suivante']]"))
                    )

                    #print("Clicking next page...")
                    driver.execute_script("arguments[0].click();", next_button)

                    WebDriverWait(driver, 2).until(EC.staleness_of(list_professionnels[0]))


                except Exception:
                    #print("no more button, last page")
                    break


        except:
            print(f"No <a> tag found inside {li.get_attribute('id')}")
